In [9]:
from pydantic import BaseModel, Field
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from dotenv import load_dotenv  # pyright: ignore[reportMissingImports]

load_dotenv(override=True)

True

In [10]:
#planner 
HOW_MANY_SEARCHES = 3

INSTRUCTIONS_PLANNER = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")
    query: str = Field(description="The search term to use for the web search.")

class WebSearchItemList(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")

planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS_PLANNER,
    model="gpt-4o-mini",
    output_type=WebSearchItemList
)

async def perform_plan(query: str) -> WebSearchItemList:
    print("Planning searches...")
    result = await Runner.run(
        planner_agent,
        f"Query: {query}",
    )
    #print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output_as(WebSearchItemList)

query = "best 2023 Phones" 

elements = await perform_plan("best 2023 Phones")
for item in elements.searches:
    print(item.query)
    print(f"\n") 

elements.searches[0].query = "test"
elements.searches[0]

Planning searches...
best smartphones of 2023


top phone reviews 2023


2023 phones comparison guide




WebSearchItem(reason='To find up-to-date rankings and reviews for the latest smartphones available in 2023.', query='test')

In [11]:
print(elements.searches[0].query)

test


In [12]:
#Evaluation
class EvaluateWebSearchItem(BaseModel):
    is_accepted: bool
    reason: str =  Field(description="why the suggestion of the AI is accepted or not")


INSTRUCTIONS_VALIDATOR = "You are an evaluator. Given a query and a WebSearchItem \
                (with its reasoning and query), decide if the search suggestion is good.\
                Return an EvaluateWebSearchItem object with fields: is_accepted (bool) \
                and reason (why the suggestion is accepted or not). \
                Use the query to judge how relevant and useful the search is."

evaluator_agent = Agent(
    name="EvaluatorAgent",
    instructions=INSTRUCTIONS_VALIDATOR,
    model="gpt-4o-mini",
    output_type=EvaluateWebSearchItem
)

async def evaluate_websearch_item(query: str, webSearchItem: WebSearchItem) -> EvaluateWebSearchItem:
    result_evaluation = await Runner.run(
        evaluator_agent,
        f"Query: {query}, WebSearchItem: {webSearchItem}"
    )
    return result_evaluation.final_output_as(EvaluateWebSearchItem)

validatedObj = await evaluate_websearch_item(query, elements.searches[0].query)
validatedObj

EvaluateWebSearchItem(is_accepted=False, reason="The WebSearchItem 'test' is too vague and does not provide any relevant information about the best phones of 2023. A more specific result is needed to be useful for the query.")

In [13]:
INSTRUCTIONS_REGENERATOR = "You are a regenerator. Given a user query,\
                a rejected WebSearchItem, and validator feedback, generate a new, more relevant\
                and precise search query that fixes the issues noted. Focus on key entities, context, and clarity. Output a WebSearchItem with: reason\
                (why this new search is important to the query) and query (the improved search term to use)."

regenerator_agent = Agent(
    name="RegeneratorAgent",
    instructions=INSTRUCTIONS_REGENERATOR,
    model="gpt-4o-mini",
    output_type=WebSearchItem
)

async def regenerator_websearch_item(query: str, rejectedItem: WebSearchItem, evaluationObject: EvaluateWebSearchItem) -> WebSearchItem:
    print("generating a new Item")
    newItem = await Runner.run(
        regenerator_agent,
        f"Query: {query}, Rejected item: {rejectedItem}, validator feedback: {evaluationObject}"
        )
    print(f"Regeneration of the prompt for the keyword result: {rejectedItem.query}")
    return newItem.final_output_as(WebSearchItem)


regenerated_object = await regenerator_websearch_item(query, elements.searches[0], validatedObj)
regenerated_object

generating a new Item
Regeneration of the prompt for the keyword result: test


WebSearchItem(reason='The new search query narrows the focus to the best smartphones of 2023, ensuring that the search results include specific rankings and reviews of current models.', query='best smartphones of 2023 reviews and rankings')

In [14]:
async def validate_websearch_item(query: str, webSearchItem: WebSearchItem) -> WebSearchItem:
    resultEvaluation = await evaluate_websearch_item(query, webSearchItem)
    EXTEND_LIMIT_OF_CALL = 0
    while not resultEvaluation.is_accepted and EXTEND_LIMIT_OF_CALL != 10:
        webSearchItem = await regenerator_websearch_item(query, webSearchItem, resultEvaluation)
        resultEvaluation = await evaluate_websearch_item(query, webSearchItem)
        EXTEND_LIMIT_OF_CALL += 1
    if EXTEND_LIMIT_OF_CALL >= 10:
        raise RuntimeError("Loop exceeded allowed iteration count (10)")
    else:
        print(f"The query {webSearchItem.query} is validated")
    return webSearchItem

await validate_websearch_item(query, elements.searches[0])

generating a new Item
Regeneration of the prompt for the keyword result: test
The query best smartphones 2023 rankings reviews is validated


WebSearchItem(reason='This new search query directly targets the request for the best smartphones in 2023, ensuring access to current rankings, reviews, and comparisons that will inform the user about their options.', query='best smartphones 2023 rankings reviews')

In [15]:
async def validate_websearch_list(query: str, webSearchItemlist: WebSearchItemList) -> WebSearchItemList:
    validatedWebSearchItemList = []
    for item in webSearchItemlist.searches:
        validatedItem = await validate_websearch_item(query, item)
        validatedWebSearchItemList.append(validatedItem)
    return WebSearchItemList(searches=validatedWebSearchItemList) 


print("Prompt of List before:")
for item in elements.searches:
    print(item.query)
    print(f"\n") 

elements = await validate_websearch_list(query, elements)

print("Prompt of List after:")
for item in elements.searches:
    print(item.query)
    print(f"\n") 

Prompt of List before:
test


top phone reviews 2023


2023 phones comparison guide


generating a new Item
Regeneration of the prompt for the keyword result: test
The query best smartphones 2023 rankings and reviews is validated
The query top phone reviews 2023 is validated
The query 2023 phones comparison guide is validated
Prompt of List after:
best smartphones 2023 rankings and reviews


top phone reviews 2023


2023 phones comparison guide




In [17]:
async def plan_deep_search(query) -> WebSearchItemList:
    webSearchItemListObject = await perform_plan(query)
    webSearchItemListObject = await validate_websearch_list(query, webSearchItemListObject)
    return webSearchItemListObject


await plan_deep_search("What are the most used AI powered Chatbots for IT Networking in 2025")

Planning searches...
The query most popular AI chatbots for IT networking 2025 is validated
The query AI chatbots for IT networking features 2025 is validated
The query customer reviews AI chatbots IT networking 2025 is validated


WebSearchItemList(searches=[WebSearchItem(reason='To find current trends and popular AI chatbots in IT networking.', query='most popular AI chatbots for IT networking 2025'), WebSearchItem(reason='To gather insights on advancements and features of IT networking chatbots.', query='AI chatbots for IT networking features 2025'), WebSearchItem(reason='To explore user reviews and industry adoption of chatbots in IT networking.', query='customer reviews AI chatbots IT networking 2025')])